In [5]:
from bs4 import BeautifulSoup as bs
import requests as req
import json


In [6]:
#ouvrir un nouveau fichier pour ecrire les donnees
file=open('data4.csv','w',encoding='utf-8')
import csv
writer=csv.writer(file)
L=["nom","date","lieu","titre","annonce",'prix',"type","autres.info","telephone","vues"]
writer.writerow(L)

65

In [7]:
#les liens des annonces dans les pages d'avito
def lien(soup):
    L=[]
    for x in soup.findAll('div',{'class':'listing listing-thumbs'}):
        for y in x.findAll('div',{'class':'item li-hover'}):
            for z in y.findAll('h2',{'class':'fs14'}):
                L.append(z.find('a')['href'])
    return L

In [8]:
#code
url="https://www.avito.ma/fr/maroc/?o="
r=req.session()
headers={'User-Agent':''}
r.headers.update(headers)
for i in range(1,17800):
    u=url+str(i)
    soup=bs(r.get(u).content,'lxml')
    L=lien(soup)
    j=0
    for j in range(len(L)):
        soup=bs(r.get(L[j]).content,'lxml')
        try:
            name=json.loads(soup.find('script', type='application/ld+json').text,strict=False)['offers']['seller']['name']#nom
            
        except:
            name='nan'
        try:
            date=soup.find('abbr',{'class':'date dtstart value'}).text.replace('\n','')#date
        except:
            date='nan'
        try:    
            lieu=soup.find('h2',{'class':'font-normal fs13 lh30 no-margin'}).text#lieu
        except:
            lieu='nan'
        try:
            titre=soup.find('h1',{'class':'page-header mbm'}).text.replace('\n','')#titre
            if("ANNONCE"in titre):
                titre=titre.split('ANNONCE')[0]
        except:
            titre='nan'
        try:
            annonce=soup.find('div',{'class':'span10'}).text.replace('\n','')#annonce
        except:
            annoce='nan'
        try:
            prix=soup.find('span',{'class':'amount value'}).text#prix
        except:
            prix='test'
        try:
            T=soup.find('aside',{'class':'panel panel-body panel-info'}).text.replace('\n',' ').split(':')
            typ=T[len(T)-1]#type
            autres=''.join(T[0:len(T)-1])# autres informations
        except:
            typ='nan'
            autres='nan'
        try:
            scripts=soup.select('script')
            scripts=(script for script in scripts)
            for script in scripts:
                if ('telephone'in script.text):
                    tele=script.text.split('telephone')[1][3:13]#telephone
        except:
            tele='nan'
        try:
            vue=soup.find('div',{'class':'panel-heading panel-heading-small'}).text.replace('\n','').split()[1]#nombre de vues
        except:
            vue='nan'
        try:
            writer.writerow([name,date,lieu,titre,annonce,prix,typ,autres,tele,vue])
        except:
            m=1
    